### Motivation
 - 클래스101에서 인기 있는 클래스의 카테고리 분석 필요
 - 빠르게 효율적으로 파악하고 정량화하기 위해 크롤링


### Lesson & Learn
 - 사용한 방법 : 속성값 별로 긁어와서 리스트를 생성한 뒤 데이터프레임으로 합치는 방식
 - 문제점 : 데이터에 Null값이 있을 경우, 리스트의 길이가 맞지 않게 됨
 - 리뷰 단위로 긁어와서 속성을 뽑아내는 방식으로 해야 함
 
 (updated : 2020-7-14)

# 1. Data Scraping

In [109]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

In [110]:
# 크롬 브라우저를 열기

DRIVER_DIR = 'driver/chromedriver'

driver = webdriver.Chrome(DRIVER_DIR)

In [111]:
# 목표 페이지 불러오기
url = "https://class101.net/search?sort=likedOrder"

driver.get(url)

In [115]:
SCROLL_PAUSE_TIME = 0.5


In [122]:
# 무한 스크롤 다운하기
# (참고 : https://xeros.dev/157)


while True:
    # 이전 높이 계산
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 화면 최하단으로 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
 
    # 페이지 로드를 기다림
    time.sleep(SCROLL_PAUSE_TIME)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight-50);")
    time.sleep(SCROLL_PAUSE_TIME)
 

    # 새로운 높이 계산
    new_height = driver.execute_script("return document.body.scrollHeight")
    time.sleep(SCROLL_PAUSE_TIME)

    # 새로운 높이가 이전 높이와 변하지 않았을 경우 스크롤 종료
    if new_height == last_height:
        break
    



In [124]:
# 전체 소스 가져오기
# (코드실행 전에 수동으로 페이지 가장 하단까지 스크롤다운 해야 함)
html = driver.page_source
driver.quit()

In [125]:
# 수업만 추출하기
bsObject = BeautifulSoup(html, 'lxml')
div_classes = bsObject.find_all('div', {'class':'sc-fzomME hhnKZs'})

print(len(div_classes))

1580


In [127]:
# 
class101 = []

for div in div_classes:
    try:
        header = div.find('div', {'class':'kPPRIZ'}).get_text()
        title = div.find('div', {'class':'sc-fzoJMP hPTfNI'}).get_text()  
        favorite = div.find('div', {'class':'CountTag__Container-sc-1bme586-0 zOfjb'}).get_text()        
        price = div.find('strong', {'class':'SellingPrice-sc-18g5z9b-0 gvjAfk'}).get_text()
        
        class101.append((header, title, favorite, price))
    except Exception:
        continue


In [130]:
# 데이터프레임으로 변환하기
import pandas as pd

df = pd.DataFrame(class101)
df.columns = ('header','title','favorite','price')

df.shape

(1396, 4)

# 2. 전처리하기 Preprocessing 

In [131]:
#카테고리와 크리에이터 분리
df['category'] = df['header'].str.split('・').str[0]
df['creator'] = df['header'].str.split('・').str[1]
df.head()

,header,title,favorite,price,category,creator
0,부업/창업/재테크・신사임당,"스마트스토어로 월 100만원 만들기, 평범한 사람이 돈을 만드는 비법",19423,"월 49,520원",부업/창업/재테크,신사임당
1,미술・둡,"연필 하나만으로 모든 분위기를 담아내요, 둡의 연필 드로잉",18477,"월 39,900원",미술,둡
2,미술・Findenart 핀든아트,"여행 드로잉, 꿈만 꾸지 마세요. 핀든아트와 여행, 그리다",16725,"월 59,900원",미술,Findenart 핀든아트
3,디지털 드로잉・반지수,"일상에 따뜻한 숨결을 불어넣어요, 반지수와 함께하는 아이패드 드로잉",15665,"월 44,900원",디지털 드로잉,반지수
4,미술・집시 zipcy,일러스트레이터 집시의 매력적인 얼굴 그리기,13633,"월 49,900원",미술,집시 zipcy


In [132]:
df.shape

(1396, 6)

In [133]:
df.tail()

,header,title,favorite,price,category,creator
1391,인테리어/소품・스몰굿띵스,폴리곤모빌 A Metallic,8,"15,000원",인테리어/소품,스몰굿띵스
1392,인테리어/소품・스몰굿띵스,폴리곤모빌 B 4종,9,"17,000원",인테리어/소품,스몰굿띵스
1393,인테리어/소품・스몰굿띵스,플랜트모빌 Gold 3종,11,"35,000원",인테리어/소품,스몰굿띵스
1394,푸드/키친・클래스101,1537 키친 클로스 2종 세트,10,"18,000원",푸드/키친,클래스101
1395,헬스/뷰티/ACC・클래스101,클라우드 요가매트 16mm 네이비,6,"32,000원",헬스/뷰티/ACC,클래스101


In [134]:
df.to_csv("class101_20200724.csv")

In [53]:
import pandas as pd
df = pd.read_csv("class101_20200714.csv", index_col=False)

In [22]:
df.head()

,Unnamed: 0,header,title,favorite,price,category,creator
0,0,부업/창업/재테크・신사임당,"스마트스토어로 월 100만원 만들기, 평범한 사람이 돈을 만드는 비법",18974,"월 53,520원",부업/창업/재테크,신사임당
1,1,미술・둡,"연필 하나만으로 모든 분위기를 담아내요, 둡의 연필 드로잉",18190,"월 39,900원",미술,둡
2,2,미술・Findenart 핀든아트,"여행 드로잉, 꿈만 꾸지 마세요. 핀든아트와 여행, 그리다",16646,"월 59,900원",미술,Findenart 핀든아트
3,3,디지털 드로잉・반지수,"일상에 따뜻한 숨결을 불어넣어요, 반지수와 함께하는 아이패드 드로잉",15509,"월 44,900원",디지털 드로잉,반지수
4,4,미술・집시 zipcy,일러스트레이터 집시의 매력적인 얼굴 그리기,13560,"월 49,900원",미술,집시 zipcy


In [54]:
df = df.drop(['Unnamed: 0', 'header'], axis=1)
df.head()

,title,favorite,price,category,creator
0,"스마트스토어로 월 100만원 만들기, 평범한 사람이 돈을 만드는 비법",18974,"월 53,520원",부업/창업/재테크,신사임당
1,"연필 하나만으로 모든 분위기를 담아내요, 둡의 연필 드로잉",18190,"월 39,900원",미술,둡
2,"여행 드로잉, 꿈만 꾸지 마세요. 핀든아트와 여행, 그리다",16646,"월 59,900원",미술,Findenart 핀든아트
3,"일상에 따뜻한 숨결을 불어넣어요, 반지수와 함께하는 아이패드 드로잉",15509,"월 44,900원",디지털 드로잉,반지수
4,일러스트레이터 집시의 매력적인 얼굴 그리기,13560,"월 49,900원",미술,집시 zipcy


In [61]:
df.shape

(903, 5)

In [65]:
# 좋아요가 500개 이상인 데이터만 추출
df_over_1000 = df[df.favorite > 1000]
df_over_1000.shape

(524, 5)

In [75]:
# 좋아요가 4300개 이상인 데이터만 추출 (상위 10%)
df_over_4300 = df[df.favorite > 4300]
df_over_4300.shape

(97, 5)

In [64]:
df['category'].isna().sum()

0

In [62]:
df['category'].value_counts()

공예           214
미술           165
디지털 드로잉      112
부업/창업/재테크     57
요리/음료         54
라이프 스타일       50
디자인/개발        48
EASY DIY      45
사진/영상         44
직무            23
음악            22
시그니처          16
굿즈            15
미술재료          10
디지털기기/ACC      7
자기계발           5
공예재료           4
악기/음악          3
운동             3
커리어            2
문구/도서          2
푸드/키친          1
헬스/뷰티/ACC      1
Name: category, dtype: int64

In [76]:
df_over_1000['category'].value_counts()

공예           133
미술           112
디지털 드로잉       83
요리/음료         40
사진/영상         31
디자인/개발        31
라이프 스타일       28
부업/창업/재테크     28
음악            12
시그니처          10
직무             9
자기계발           2
EASY DIY       2
미술재료           1
운동             1
디지털기기/ACC      1
Name: category, dtype: int64

In [77]:
df_over_4300['category'].value_counts()

디지털 드로잉      26
미술           23
공예           15
사진/영상         9
부업/창업/재테크     8
디자인/개발        6
요리/음료         6
라이프 스타일       2
시그니처          1
음악            1
Name: category, dtype: int64

In [55]:
# 전체 강의 중 디지털드로잉 추출
df_ddrawing = df[df['category'] == '디지털 드로잉']
df_ddrawing.shape

(112, 5)

In [40]:
df_ddrawing.head(10)

,header,title,favorite,price,category,creator
3,디지털 드로잉・반지수,"일상에 따뜻한 숨결을 불어넣어요, 반지수와 함께하는 아이패드 드로잉",15509,"월 44,900원",디지털 드로잉,반지수
17,디지털 드로잉・그리지영,나만의 문방구를 차려요! 그리지영의 타블렛으로 굿즈 만들기,7770,"월 30,400원",디지털 드로잉,그리지영
18,디지털 드로잉・사심굿즈,취미를 넘어 굿즈 셀러로! 인쇄물 굿즈 제작과 상품화,7669,"월 39,900원",디지털 드로잉,사심굿즈
21,디지털 드로잉・그림바지,"새로운 드로잉 기법의 설렘, 레트로 픽셀아트 드로잉",7277,"월 31,960원",디지털 드로잉,그림바지
25,디지털 드로잉・그림비,일러스트레이터 배성태(그림비)의 달달한 일상드로잉,7201,"월 42,720원",디지털 드로잉,그림비
28,디지털 드로잉・별라랜드,"차근차근, 아이패드로 그리는 미니멀한 손그림 일러스트",6952,"월 42,500원",디지털 드로잉,별라랜드
38,디지털 드로잉・배성규,배성규 작가와 함께 아이패드로 그리는 스물네 번의 보름,6172,"월 51,880원",디지털 드로잉,배성규
40,디지털 드로잉・우동,소중한 기억에 나만의 색을 칠하다 - 일러스트레이터 우동의 디지털 페인팅,6057,"월 46,150원",디지털 드로잉,우동
41,디지털 드로잉・캣빵,아이패드로 시작하는 캣빵의 귀여운 굿즈 만들기,6039,"월 43,880원",디지털 드로잉,캣빵
43,디지털 드로잉・키츠 (kits.),아이패드 하나로 떠나는 키츠의 아날로그 텍스처 여행,5770,"월 24,880원",디지털 드로잉,키츠 (kits.)


## 수익, 굿즈, 이모티콘 관련 클래스 추출하기 

In [78]:
df[df['title'].str.contains('수익')]

,title,favorite,price,category,creator
10,블로그로 하루 2시간 투자해서 직장인만큼 수익내기!,9226,"월 45,632원",부업/창업/재테크,리뷰요정리남
34,"15만 유튜버 자청의 자동수익 월 1,000만원 만드는 법",6443,"월 111,298원",부업/창업/재테크,라이프 해커 자청
83,월세 수익 300만 원을 만드는 송사무장의 부동산 투자법!,4474,"월 27,740원",부업/창업/재테크,송사무장
181,"월 100만원 수익! 아이디어스, 스마트스토어에서 월급 외 수익 벌기",3035,"월 43,058원",부업/창업/재테크,단아쌤
505,쿠팡 파트너스와 제휴 마케팅으로 집에서 월 200만 원 부수익 만들기,1069,"월 43,058원",부업/창업/재테크,1프로마케터 서동민
542,만화로 수익 창출까지! 난희의 아이패드로 SNS 웹툰 그려보기,937,"월 46,440원",디지털 드로잉,난희
544,"소소하지만 확실하게, 이모티콘으로 수익 창출하기",934,"월 34,900원",디지털 드로잉,수팡
567,"프리랜서로 N개의 직업을 가지며, 나만의 수익을 만드는 방법!",865,"월 48,848원",부업/창업/재테크,코스모지나
575,내 스마트스토어 상품을 해외에 파는 법 : 쇼피파이로 매일 2시간 투자해서 수익 불리기!,838,"월 52,984원",부업/창업/재테크,온도남
599,"연봉 1억 유튜버가 도와주는 유튜버 되는 지름길, 채널 생성부터 수익화까지!",778,"월 49,758원",부업/창업/재테크,구스마일


In [87]:
# 해당되는 클래스의 갯수
df['title'].str.contains('수익').sum()

14

In [88]:
df[df['title'].str.contains('굿즈')]

,title,favorite,price,category,creator
17,나만의 문방구를 차려요! 그리지영의 타블렛으로 굿즈 만들기,7770,"월 30,400원",디지털 드로잉,그리지영
18,취미를 넘어 굿즈 셀러로! 인쇄물 굿즈 제작과 상품화,7669,"월 39,900원",디지털 드로잉,사심굿즈
41,아이패드로 시작하는 캣빵의 귀여운 굿즈 만들기,6039,"월 43,880원",디지털 드로잉,캣빵
71,일러스트레이터 이공과 함께 나만의 미니 굿즈 제작하기,4907,"월 35,900원",디지털 드로잉,이공
121,"소소하지만 확실한 행복, 유르의 굿즈 제작부터 판매까지",3905,"월 36,900원",디지털 드로잉,유르
155,"실로 그리는 그림, 스왈로우굿즈의 일러스트 프랑스자수",3365,"월 46,900원",공예,스왈로우굿즈
186,전구별과 떠나는 캐릭터 제작과 굿즈 만들기 세상,2996,"월 43,046원",디지털 드로잉,전구별
199,별별그림의 실크스크린으로 쓱싹 만드는 유니크한 굿즈,2876,"월 43,900원",공예,별별그림
213,귀염뽀짝 일러스트 굿즈 만들기의 모든 것!,2794,"월 43,880원",디지털 드로잉,뽐뽐 스튜디오
284,내가 그린 캐릭터로 브랜드 시작하기! 굿즈부터 사업화까지,2169,"월 49,676원",부업/창업/재테크,효샤


### '굿즈' 키워드 포함하는 클래스

In [89]:
df[df['title'].str.contains('굿즈')]

,title,favorite,price,category,creator
17,나만의 문방구를 차려요! 그리지영의 타블렛으로 굿즈 만들기,7770,"월 30,400원",디지털 드로잉,그리지영
18,취미를 넘어 굿즈 셀러로! 인쇄물 굿즈 제작과 상품화,7669,"월 39,900원",디지털 드로잉,사심굿즈
41,아이패드로 시작하는 캣빵의 귀여운 굿즈 만들기,6039,"월 43,880원",디지털 드로잉,캣빵
71,일러스트레이터 이공과 함께 나만의 미니 굿즈 제작하기,4907,"월 35,900원",디지털 드로잉,이공
121,"소소하지만 확실한 행복, 유르의 굿즈 제작부터 판매까지",3905,"월 36,900원",디지털 드로잉,유르
155,"실로 그리는 그림, 스왈로우굿즈의 일러스트 프랑스자수",3365,"월 46,900원",공예,스왈로우굿즈
186,전구별과 떠나는 캐릭터 제작과 굿즈 만들기 세상,2996,"월 43,046원",디지털 드로잉,전구별
199,별별그림의 실크스크린으로 쓱싹 만드는 유니크한 굿즈,2876,"월 43,900원",공예,별별그림
213,귀염뽀짝 일러스트 굿즈 만들기의 모든 것!,2794,"월 43,880원",디지털 드로잉,뽐뽐 스튜디오
284,내가 그린 캐릭터로 브랜드 시작하기! 굿즈부터 사업화까지,2169,"월 49,676원",부업/창업/재테크,효샤


In [92]:
df[df['title'].str.contains('굿즈')].category.value_counts()

디지털 드로잉      9
부업/창업/재테크    2
공예           2
디자인/개발       1
굿즈           1
Name: category, dtype: int64

In [58]:
df_ddrawing[df_ddrawing['title'].str.contains('이모티콘')]

,title,favorite,price,category,creator
48,민두 작가의 이모티콘 제작부터 판매까지!,5681,"월 31,880원",디지털 드로잉,화유
90,광팔의 아이패드로 귀염통통 카카O톡 이모티콘 만들기,4368,"월 32,780원",디지털 드로잉,광팔
115,식빵뚱냥과 귀염뽀짝 캐릭터 이모티콘 만들기,4080,"월 38,900원",디지털 드로잉,식빵뚱냥
246,"모찌, 세숑, 바쁘개 이모티콘은 어떻게 만들었을까? 이모티콘 제작 입문!",2397,"월 45,900원",디지털 드로잉,이모티콘 좋아요
317,이모티콘 작가 오니기리와 함께 카카오톡 이모티콘 만들기,1946,"월 34,900원",디지털 드로잉,오니기리
371,"귀여운 낙서로 돈 버는 법, 캐릭터로 이모티콘 연금 만들기!",1699,"월 51,440원",디지털 드로잉,김하나
544,"소소하지만 확실하게, 이모티콘으로 수익 창출하기",934,"월 34,900원",디지털 드로잉,수팡


In [60]:
df[df['title'].str.contains('굿즈')]

,title,favorite,price,category,creator
17,나만의 문방구를 차려요! 그리지영의 타블렛으로 굿즈 만들기,7770,"월 30,400원",디지털 드로잉,그리지영
18,취미를 넘어 굿즈 셀러로! 인쇄물 굿즈 제작과 상품화,7669,"월 39,900원",디지털 드로잉,사심굿즈
41,아이패드로 시작하는 캣빵의 귀여운 굿즈 만들기,6039,"월 43,880원",디지털 드로잉,캣빵
71,일러스트레이터 이공과 함께 나만의 미니 굿즈 제작하기,4907,"월 35,900원",디지털 드로잉,이공
121,"소소하지만 확실한 행복, 유르의 굿즈 제작부터 판매까지",3905,"월 36,900원",디지털 드로잉,유르
155,"실로 그리는 그림, 스왈로우굿즈의 일러스트 프랑스자수",3365,"월 46,900원",공예,스왈로우굿즈
186,전구별과 떠나는 캐릭터 제작과 굿즈 만들기 세상,2996,"월 43,046원",디지털 드로잉,전구별
199,별별그림의 실크스크린으로 쓱싹 만드는 유니크한 굿즈,2876,"월 43,900원",공예,별별그림
213,귀염뽀짝 일러스트 굿즈 만들기의 모든 것!,2794,"월 43,880원",디지털 드로잉,뽐뽐 스튜디오
284,내가 그린 캐릭터로 브랜드 시작하기! 굿즈부터 사업화까지,2169,"월 49,676원",부업/창업/재테크,효샤
